# Creating Project

In [ ]:
import warnings

warnings.filterwarnings("ignore")

from datetime import datetime
import mitzu.model as M


p = M.Project(
    project_name="trino_test_project",
    event_data_tables=[
        M.EventDataTable.create(
            table_name="sub_events",
            schema="tiny",
            event_name_alias="user_subscribe",
            event_time_field="subscription_time",
            user_id_field="subscriber_id",
        ),
        M.EventDataTable.create(
            table_name="web_events",
            schema="tiny",
            event_name_field="event_name",
            event_time_field="event_time",
            date_partition_field="event_time",
            user_id_field="user_id",
            event_specific_fields=["event_properties"],
        ),
    ],
    discovery_settings=M.DiscoverySettings(
        end_dt=datetime(2021, 4, 1),
        lookback_days=10,
        property_sample_rate=10,
    ),
    webapp_settings=M.WebappSettings(
        end_date_config=M.WebappEndDateConfig.CUSTOM_DATE,
        custom_end_date=datetime(2021, 4, 1),
    ),
    connection=M.Connection(
        connection_name="trino_hive_connection",
        connection_type=M.ConnectionType.TRINO,
        user_name="test",
        secret_resolver=None,
        catalog="minio",
        host="localhost",
    ),
)

# Saving Project to File

In [ ]:
dp = p.discover_project()
dp.save_to_project_file("trino_test_project")

# Loading Project

In [ ]:
dp = M.DiscoveredProject.load_from_project_file("trino_test_project")
m = dp.create_notebook_class_model()

# Product Analytics From Loaded Project

In [ ]:
(m.page_visit >> m.search).config(
    conv_window="7 days", time_group="total", group_by=m.page_visit.event_properties.url
).print_sql()

In [ ]:
m.page_visit.event_properties.url.is_www_awestore_com.config(chart_type="line")

In [ ]:
(m.checkout.event_properties.price_shown > 100).config(chart_type="line")

In [ ]:
(m.page_visit >> m.checkout).config(
    time_group="total",
    conv_window="3 week",
    group_by=m.page_visit.user_properties.country_code,
    lookback_days=120,
)

In [ ]:
(m.page_visit >> m.checkout).config(
    time_group="total",
    conv_window="3 week",
    group_by=m.page_visit.user_properties.country_code,
    lookback_days=120,
).print_sql()

In [ ]:
# dp.display_inline_dashboard(port=8083)

# or when your notebook does not support iframes
# dp.display_external_dashboard(port=8083)